In [1]:
# %matplotlib inline

import warnings
import numpy as np
import pandas as pd
import scipy.stats as st
import statsmodels as sm
import matplotlib
import glob, os    
import matplotlib.pyplot as plt
from scipy.stats._continuous_distns import _distn_names
from cycler import cycler

matplotlib.rcParams['figure.figsize'] = (16.0, 12.0)
matplotlib.style.use('ggplot')

matplotlib.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

In [2]:
datapath="C:/Users/kbada/OneDrive/Desktop/github/forgedMachines"
demanddata = pd.concat(map(pd.read_csv, glob.glob(os.path.join(datapath, "DeliveryData/DeliveryData*.csv"))))
demanddata.drop(columns=["Unnamed: 0"],inplace=True)
demanddata.columns
# zip
zip_codes=pd.concat(map(pd.read_csv, glob.glob(os.path.join(datapath, "ZIP CODE.csv"))))
zip_codes=list(zip_codes.columns)
zip_codes=[int(i) for i in zip_codes]
zipdemand=demanddata.groupby(['customer ZIP','scenario']).agg({'quantity':'sum'})
zipdemand.reset_index(inplace=True)
# commodity demand
commodity_demand=demanddata.groupby(['origin facility','customer ZIP','scenario']).agg({'quantity':'sum'})
commodity_demand.reset_index(inplace=True)

In [3]:
print(_distn_names)

['ksone', 'kstwobign', 'norm', 'alpha', 'anglit', 'arcsine', 'beta', 'betaprime', 'bradford', 'burr', 'burr12', 'fisk', 'cauchy', 'chi', 'chi2', 'cosine', 'dgamma', 'dweibull', 'expon', 'exponnorm', 'exponweib', 'exponpow', 'fatiguelife', 'foldcauchy', 'f', 'foldnorm', 'weibull_min', 'weibull_max', 'frechet_r', 'frechet_l', 'genlogistic', 'genpareto', 'genexpon', 'genextreme', 'gamma', 'erlang', 'gengamma', 'genhalflogistic', 'gompertz', 'gumbel_r', 'gumbel_l', 'halfcauchy', 'halflogistic', 'halfnorm', 'hypsecant', 'gausshyper', 'invgamma', 'invgauss', 'norminvgauss', 'invweibull', 'johnsonsb', 'johnsonsu', 'laplace', 'levy', 'levy_l', 'levy_stable', 'logistic', 'loggamma', 'loglaplace', 'lognorm', 'gilbrat', 'maxwell', 'mielke', 'kappa4', 'kappa3', 'moyal', 'nakagami', 'ncx2', 'ncf', 't', 'nct', 'pareto', 'lomax', 'pearson3', 'powerlaw', 'powerlognorm', 'powernorm', 'rdist', 'rayleigh', 'reciprocal', 'rice', 'recipinvgauss', 'semicircular', 'skewnorm', 'trapz', 'triang', 'truncexpon',

Approach 1:

In [80]:
# Using website: https://pythonhealthcare.org/2018/05/03/81-distribution-fitting-to-data/
import warnings
import scipy
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore")

def dist_fitter(y):
    sc = StandardScaler() 
    y = y.values.reshape(-1, 1)
    sc.fit(y)
    y = sc.transform(y)
    y = y.flatten()
    size = len(y)

    dist_names = ['expon', 'lognorm', 'norm']
    
#     dist_names = ['beta',
#                   'expon',
#                   'gamma',
#                   'lognorm',
#                   'norm',
#                   'pearson3',
#                   'triang',
#                   'uniform',
#                   'weibull_min', 
#                   'weibull_max']


    chi_square = []
    p_values = []

    # Set up 50 bins for chi-square test
    # Observed data will be approximately evenly distrubuted aross all bins
    percentile_bins = np.linspace(0, 100, 10)
    percentile_cutoffs = []
    percentile_cutoffs = np.percentile(y, percentile_bins, interpolation='midpoint')
    
    for i in range(len(percentile_cutoffs)-1):
        diff = percentile_cutoffs[i+1] - percentile_cutoffs[i]
        if diff <= 0:
            percentile_cutoffs[i+1] = percentile_cutoffs[i+1] - diff + 0.00000001
    observed_frequency, bins = (np.histogram(y, bins=percentile_cutoffs))
    cum_observed_frequency = np.cumsum(observed_frequency)

    # Loop through candidate distributions

    for distribution in dist_names:
        # Set up distribution and get fitted distribution parameters
        dist = getattr(scipy.stats, distribution)
        param = dist.fit(y)

        # Obtain the KS test P statistic, round it to 5 decimal places
        p = scipy.stats.kstest(y, distribution, args=param)[1]
        p = np.around(p, 5)
        p_values.append(p)    

        # Get expected counts in percentile bins
        # This is based on a 'cumulative distrubution function' (cdf)
        cdf_fitted = dist.cdf(percentile_cutoffs, *param[:-2], loc=param[-2], 
                              scale=param[-1])
        expected_frequency = []
        for bin in range(len(percentile_bins)-1):
            expected_cdf_area = cdf_fitted[bin+1] - cdf_fitted[bin]
            expected_frequency.append(expected_cdf_area)

        # calculate chi-squared
        expected_frequency = np.array(expected_frequency) * size
        cum_expected_frequency = np.cumsum(expected_frequency)
        ss = sum(((cum_expected_frequency - cum_observed_frequency) ** 2) / cum_observed_frequency)
        chi_square.append(ss)

    # Collate results and sort by goodness of fit (best at top)
    results = pd.DataFrame()
    results['Distribution'] = dist_names
    results['chi_square'] = chi_square
    results['p_value'] = p_values
    results.sort_values(['chi_square'], inplace=True)
    
    return results

In [81]:
def get_dist_params(y, best_distribution):
    # Create an empty list to stroe fitted distribution parameters
    parameters = []
    num_distributions = 1

    # Loop through the distributions ot get line fit and paraemters
    dist_name = best_distribution['Distribution'].values.item()
    dist = getattr(scipy.stats, dist_name)
    param = dist.fit(y)
    parameters = param

    # Store distribution paraemters in a dataframe (this could also be saved)
    best_distribution['Distribution parameters'] = [param]

    return best_distribution

In [85]:
counter = 0
num_distributions = 1
od_distributions = pd.DataFrame()

# Just get first one
for zipcode in zip_codes:    
    for ori_node in set(commodity_demand['origin facility']):
        # Get demand data for each OD-pair
        od_demand = commodity_demand.loc[np.logical_and(commodity_demand['customer ZIP'] == zipcode, commodity_demand['origin facility'] == ori_node)]
        data = od_demand['quantity']
        results = dist_fitter(data)
        name = str(ori_node) + "-" + str(zipcode)
        best_dist = results.iloc[0:num_distributions]
        dist_names = results['Distribution'].iloc[0:num_distributions].values.item()
        dist_with_params = get_dist_params(data, best_dist)
        # Populate dataframe with results
        od_distributions = od_distributions.append({'ZIP': zipcode, 'Origin': ori_node, 
                                                    'Distribution': dist_with_params['Distribution'].iloc[0], 
                                                    'Parameters': dist_with_params['Distribution parameters'].iloc[0],
                                                    'Chi-square': dist_with_params['chi_square'].iloc[0],
                                                    'P-value': dist_with_params['p_value'].iloc[0]}, ignore_index=True)
od_distributions = od_distributions[['Origin', 'ZIP', 'Distribution', 'Parameters', 'Chi-square', 'P-value']]
od_distributions = od_distributions.astype({'ZIP': 'int32'})
od_distributions.to_csv('od_distributions.csv', index=False)